## 3D Matrices Graph

A more complex model with a 3-Dimensional input and output layer

## Build and save the graph

Ensure you are working in a virtual environment with tensorflow >= 1.9 installed, and in the notebook's directory on the command line:

```bash
rm -r saved_graph
```

In [1]:
import tensorflow as tf

In [2]:
# construct the graph

X = tf.placeholder(tf.float32, shape=(3,3,3), name="input")

X_0 = tf.add(X[0,:,:], 1)
X_1 = tf.add(X[1,:,:], 2)
X_2 = tf.add(X[2,:,:], 3)

# Z = tf.stack([X_0, X_1, X_2], name="output_z") # tf.stack is not supported by tensorflow lite

axis = 0
Z = tf.concat([tf.expand_dims(t, axis) for t in [X_0, X_1, X_2]], axis, name="output")

init = tf.global_variables_initializer()

X_feed = [[[1,  2,  3],  [4,  5,  6],  [7,  8,  9]],
          [[10, 20, 30], [40, 50, 60], [70, 80, 90]],
          [[100,200,300],[400,500,600],[700,800,900]]]

with tf.Session() as sess:
    sess.run(init)
    z_eval = sess.run([Z], feed_dict={X: X_feed})
    
    tf.saved_model.simple_save(sess, "./saved_graph", inputs={"input": X}, outputs={"output": Z})
    
    print(z_eval)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_graph/saved_model.pb
[array([[[  2.,   3.,   4.],
        [  5.,   6.,   7.],
        [  8.,   9.,  10.]],

       [[ 12.,  22.,  32.],
        [ 42.,  52.,  62.],
        [ 72.,  82.,  92.]],

       [[103., 203., 303.],
        [403., 503., 603.],
        [703., 803., 903.]]], dtype=float32)]


In [3]:
# Some test code

sess = tf.InteractiveSession()

T = tf.constant([[[1,  2,  3],  [4,  5,  6],  [7,  8,  9]],
                 [[10, 20, 30], [40, 50, 60], [70, 80, 90]],
                 [[100,200,300],[400,500,600],[700,800,900]]], shape=(3,3,3))

T_0 = tf.add(T[0,:,:], 1)
T_1 = tf.add(T[1,:,:], 2)
T_2 = tf.add(T[2,:,:], 3)

# T_out = tf.stack([T_0, T_1, T_2])

axis = 0
T_out = tf.concat([tf.expand_dims(t, axis) for t in [T_0, T_1, T_2]], axis)

print( "T_0:", "\n", T_0.eval(), "\n" )
print( "T_1:", "\n", T_1.eval(), "\n" )
print( "T_2:", "\n", T_2.eval(), "\n" )
print( "T_out:", "\n", T_out.eval() )

sess.close()

T_0: 
 [[ 2  3  4]
 [ 5  6  7]
 [ 8  9 10]] 

T_1: 
 [[12 22 32]
 [42 52 62]
 [72 82 92]] 

T_2: 
 [[103 203 303]
 [403 503 603]
 [703 803 903]] 

T_out: 
 [[[  2   3   4]
  [  5   6   7]
  [  8   9  10]]

 [[ 12  22  32]
  [ 42  52  62]
  [ 72  82  92]]

 [[103 203 303]
  [403 503 603]
  [703 803 903]]]


## Freeze The Graph

Having used the `tf.saved_model.simple_save` function, now use the `freeze_graph` cli. Tensorflow also provides a python interface to the freeze_graph function.

On the command line:

```bash
mkdir frozen_graph

freeze_graph \
    --input_saved_model_dir=saved_graph \
    --saved_model_tags=serve \
    --output_graph=frozen_graph/frozen_graph.pb \
    --output_node_names=output
```

## Convert the Model to a TensorFlow Lite Format

Use the `toco` command line utility to convert the frozen graph to a tensorflow lite compatible file:

```bash
mkdir tflite

toco \
	--graph_def_file=frozen_graph/frozen_graph.pb \
	--output_file=tflite/model.tflite \
	--input_arrays=input \
	--input_shapes=3,3,3 \
	--output_arrays=output
```